<a href="https://colab.research.google.com/github/Edna-cyber/telepresence/blob/3d/3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("content/drive")